In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import re
# import nltk
from sklearn.model_selection import train_test_split
# from nltk.corpus import stopwords
import joblib

import stopwordsiso as stopwords



In [3]:
df = pd.read_csv('../dataset/sentences.csv',  delimiter=',', quoting=2, encoding="utf-8")

In [4]:
df.head(5)

,id,tweets,labels
0,0.0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1.0,"Try talking with ChatGPT, our new AI system wh...",good
2,2.0,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3.0,"THRILLED to share that ChatGPT, our new model ...",good
4,4.0,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [14]:
stpwrd = stopwords.stopwords("en")

In [19]:
def procces_sentence(review:str)->str:
    sample_review = re.sub(r'http\S+', '', review)
    sample_review = re.sub("[^a-zA-Z]",' ',sample_review)
    sample_review = sample_review.lower()
    sample_review = sample_review.split()
    swords = set(stpwrd)                     
    sample_review = [w for w in sample_review if w not in swords and len(w) > 1]
    sample_review = " ".join(sample_review)
    return sample_review

def take_res(count:int)->int:
    labels = df["labels"][count]
    if labels == "bad": return 0
    elif labels == "neutral": return 1
    elif labels == "good": return 2


: 

In [27]:
from tqdm import tqdm

count_t = 219288

trainx = []
trainy = []
for i in tqdm(range(count_t)):
    trainx.append(procces_sentence(df["tweets"][i]))
    trainy.append(take_res(i))



100%|██████████| 219288/219288 [00:17<00:00, 12427.64it/s]


In [6]:
df2 = pd.DataFrame()

df2['sentence'] = pd.DataFrame(trainx)
df2['status'] = pd.DataFrame(trainy)

In [14]:
df2.to_csv('dataset/proccessed.csv')

In [28]:
trainx

['chatgpt optimizing language models dialogue openai',
 'talking chatgpt optimized dialogue feedback improve',
 'chatgpt optimizing language models dialogue machinelearning datascience artificialintelligence ntrending article identified amp digested granola machine driven rss bot ramsey elbasheer',
 'thrilled share chatgpt model optimized dialog public accessible',
 'minutes openai released chatgpt nand',
 'launched chatgpt optimized dialogue',
 'minutes openai released chatgpt nand',
 'chatgpt coming strong refusing stalk agreeing providing waldo',
 'penal deployed helping build couple months chatbot based gpt excited share vl',
 'preview model chatgpt nwe preview waitlist',
 'god damn openai announcing busy',
 'openai chatgpt optimizing language models dialogue',
 'chatgpt optimizing language models dialogue machinelearning datascience artificialintelligence ntrending article identified amp digested granola machine driven rss bot ramsey elbasheer',
 'openai announced chatgpt model op

In [7]:
"""
with open("trainx.pkl", "wb") as file:
    joblib.dump(trainx, file)
with open("trainy.pkl", "wb") as file:
    joblib.dump(trainy, file)
"""



In [32]:
trainy = np.array(trainy)
trainy
train_x, test_x, y_train, y_test = train_test_split(trainx,trainy, test_size = 0.1)
vectorizer = CountVectorizer( max_features = 40000 )
train_x = vectorizer.fit_transform(train_x)



In [30]:
train_x.shape, y_train.shape

((197359, 40000), (197359,))

In [17]:
model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
model.fit(train_x, y_train)

RandomForestClassifier(random_state=42)

In [33]:
test_xx = vectorizer.transform(test_x)
test_xx
test_xx = test_xx.toarray()

In [ ]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
import pandas as pd
import numpy as np

# Modelleri ekleyelim
model_r = {
    # Regresyon Modelleri

    # 'Random Forest Regressor': RandomForestRegressor(),
    # 'Extra Trees Regressor': ExtraTreesRegressor(),
    'AdaBoost Regressor': AdaBoostRegressor(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'K Neighbors Regressor': KNeighborsRegressor(),
    'CatBoost Regressor': CatBoostRegressor(learning_rate=0.1, depth=6, iterations=500, verbose=0),
    

}

model_c = {
        # Sınıflandırma Modelleri
    'Logistic Regression': LogisticRegression(),
    'Perceptron': Perceptron(),
    'Random Forest Classifier': RandomForestClassifier(n_estimators=100),
    'Extra Trees Classifier': ExtraTreesClassifier(),
    'Decision Tree Classifier': DecisionTreeClassifier(),
    'Extreme Gradient Boosting': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'CatBoost Classifier': CatBoostClassifier(learning_rate=0.1, depth=6, iterations=500, verbose=0)
}
# Sonuçları ayrı ayrı kaydedelim
results_regression = {}
results_classification = {}

# Regresyon Modellerini Çalıştır
# for name, model in model_r.items():
#     model.fit(train_x, y_train)
#     y_pred = model.predict(test_xx)
#     rmse = np.sqrt(mean_squared_error(y_test, y_pred))
#     results_regression[name] = {'RMSE': rmse}
#     print(f'{name}: RMSE = {rmse}')

# Sınıflandırma Modellerini Çalıştır
for name, model in model_c.items():
    model.fit(train_x, y_train)
    y_pred = model.predict(test_xx)
    accuracy = accuracy_score(y_test, y_pred)
    results_classification[name] = {'Accuracy': accuracy}
    print(f'{name}: Accuracy = {accuracy * 100:.2f}%')

# Sonuçları DataFrame'e Çevir
results_regression_df = pd.DataFrame(results_regression).T
results_classification_df = pd.DataFrame(results_classification).T

# Sonuçları yazdır
print("Regression Results:")
print(results_regression_df)
print("\nClassification Results:")
print(results_classification_df)


C:\Users\ahayd\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression: Accuracy = 75.06%
Perceptron: Accuracy = 67.07%


In [19]:
from sklearn.metrics import accuracy_score

# Test seti üzerinde tahmin yap
y_pred = model.predict(test_xx)

# Doğruluğu hesapla
accuracy = accuracy_score(y_test, y_pred)

# Sonucu yazdır
print(f"Modelin doğruluk oranı: {accuracy * 100:.2f}%")


Modelin doğruluk oranı: 81.89%


In [ ]:
def predict_sentiment(new_sentence, model, vectorizer):
    cleaned_sentence = procces_sentence(new_sentence)
    new_sentence_vector = vectorizer.transform([cleaned_sentence])
    new_sentence_vector = new_sentence_vector.toarray()
    prediction = model.predict(new_sentence_vector)
    
    if prediction == 2:
        sentiment = "positive"
    elif prediction == 1: sentiment = "neutral"
    else:
        sentiment = "negative"
    
    return sentiment




In [ ]:
new_sentence = "you are so good and in love"

predicted_sentiment = predict_sentiment(new_sentence, model, vectorizer)
print(f"Girilen cümlenin sentiment değeri: {predicted_sentiment}")

In [ ]:


#with open("trained_model.pkl", "wb") as file:
 #   joblib.dump(model, file)